<a href="https://colab.research.google.com/github/vamsi-pillala/sample_repo1/blob/main/testing_apache_beam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import apache_beam  # Import the module

# Get all attributes (including methods)
all_attributes = dir(apache_beam)

# Print all methods starting with lowercase (assuming most methods are functions)
for attr in all_attributes:
  if not attr.startswith('__') and callable(getattr(apache_beam, attr)):
    print(attr)

In [ ]:
import apache_beam as beam

def print_hello(element):
    print("Hello, World!")

def run():
    # Create a Pipeline object
    with beam.Pipeline() as pipeline:
        # Create a PCollection with a single element
        data = pipeline | 'Create' >> beam.Create([1])

        # Apply a ParDo transform to print "Hello, World!"
         | 'PrintHello' >> beam.ParDo(print_hello)

if __name__ == '__main__':
    run()


Hello, World!


In [ ]:
!{'pip install apache_beam[gcp]'}

In [ ]:
with open('sample_data/card_skippers-00000-of-00001') as file:
  points_wise_list = []
  for line in file:
    customer_id,key_name,points = line.split(',')
    points = points.rstrip().lstrip()
    points,chr1,chr2 = points.split(' ')
    temp_tuple = (customer_id+','+key_name,points)
    points_wise_list.append(temp_tuple)
  sorted_points_wise_list = sorted(points_wise_list,key = lambda x:x[1],reverse = True)

  print('Credit Defaulter List')

  for record in sorted_points_wise_list:
    composite_name,points = record
    print(composite_name + ' ' + str(points) + ' ' + 'fraud points')

In [ ]:
import apache_beam as beam
p = beam.Pipeline()

def calculate_points(element):
  customer_id,first_name,last_name,relationship_id,card_type,max_limit,spent,cash_withdrawn,payment_cleared,payment_date = element.split(',')

  spent = int(spent)
  payment_cleared = int(payment_cleared)
  max_limit = int(max_limit)

  key_name = customer_id + ', ' + first_name + ' ' + last_name
  defaulter_points = 0

  if payment_cleared < 0.7*spent:
    defaulter_points += 1
  if (payment_cleared == max_limit) and (payment_cleared < spent):
    defaulter_points += 1
  if (payment_cleared == max_limit) and (payment_cleared < 0.7*spent):
    defaulter_points += 1

  return key_name,defaulter_points

def format_result(sum_pair):
  key_name,points = sum_pair
  return key_name + ', ' + str(points) + ' fraud points'

card_defaulter = (
    p
    |beam.io.ReadFromText('cards.txt',skip_header_lines=1)
    |beam.Map(calculate_points)
    |beam.CombinePerKey(sum)
    |beam.Filter(lambda element:element[1] > 0)
    |beam.Map(format_result)
    |beam.io.WriteToText('sample_data/card_skippers')
)

p.run()

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()

input_Pcollection = (
    p1
    |'read from text file' >> beam.io.ReadFromText('pipe_delimited_cards_data_sample.txt',skip_header_lines=True)
    |'splitting records as list items' >> beam.Map(lambda record:record.split('|'))
)
chunk_of_data_based_on_card_type = (
    input_Pcollection
    |'filtering records based on card type' >> beam.Filter(lambda record:record[4] == 'Issuers')
    |'string formatting card-type based records' >> beam.Map(lambda record:record[0] + ', ' + record[1] + ' ' + record[2] + ', ' + record[3] + ', ' + record[4] + ', ' + record[5] + ', ' + record[6] + ', ' + record[7] + ', ' + record[8] + ', ' + record[9])
    |'writing results for issuers card-type customers to a file' >> beam.io.WriteToText('sample_data/issuers_card_holders')
)

chunk_of_data_based_on_year_of_last_date = (
    input_Pcollection
    |'filtering records based on year on last-date and less than half due amount paid' >> beam.Filter(lambda record:record[9][-4:] == '2019' and int(record[8]) < 0.5*int(record[5]))
    |'string formatting year on last-date and less than half due amount paid based records' >> beam.Map(lambda record:record[0] + ', ' + record[1] + ' ' + record[2] + ', ' + record[3] + ', ' + record[4] + ', ' + record[5] + ', ' + record[6] + ', ' + record[7] + ', ' + record[8] + ', ' + record[9])
    |'writing results for 2019 year last date and less than half amount paid customers to a file' >> beam.io.WriteToText('sample_data/2019_less_than_half_payers')
)
p1.run()

!{('head -n 100 sample_data/2019_less_than_half_payers-00000-of-00001')}

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!{'pip install apache_beam'}